## [Computational Social Science] Project 5: Natural Language Processing

In this project, you will use natural language processing techniques to explore a dataset containing tweets from members of the 116th United States Congress that met from January 3, 2019 to January 2, 2021. The dataset has also been cleaned to contain information about each legislator. Concretely, you will do the following:

* Preprocess the text of legislators' tweets
* Conduct Exploratory Data Analysis of the text
* Use sentiment analysis to explore differences between legislators' tweets
* Featurize text with manual feature engineering, frequency-based, and vector-based techniques
* Predict legislators' political parties and whether they are a Senator or Representative

You will explore two questions that relate to two central findings in political science and examine how they relate to the text of legislators' tweets. First, political scientists have argued that U.S. politics is currently highly polarized relative to other periods in American history, but also that the polarization is asymmetric. Historically, there were several conservative Democrats (i.e. "blue dog Democrats") and liberal Republicans (i.e. "Rockefeller Republicans"), as measured by popular measurement tools like [DW-NOMINATE](https://en.wikipedia.org/wiki/NOMINATE_(scaling_method)#:~:text=DW\%2DNOMINATE\%20scores\%20have\%20been,in\%20the\%20liberal\%2Dconservative\%20scale.). However, in the last few years, there are few if any examples of any Democrat in Congress being further to the right than any Republican and vice versa. At the same time, scholars have argued that this polarization is mostly a function of the Republican party moving further right than the Democratic party has moved left. **Does this sort of asymmetric polarization show up in how politicians communicate to their constituents through tweets?**

Second, the U.S. Congress is a bicameral legislature, and there has long been debate about partisanship in the Senate versus the House. The House of Representatives is apportioned by population and all members serve two year terms. In the Senate, each state receives two Senators and each Senator serves a term of six years. For a variety of reasons (smaller chamber size, more insulation from the voters, rules and norms like the filibuster, etc.), the Senate has been argued to be the "cooling saucer" of Congress in that it is more bipartisan and moderate than the House. **Does the theory that the Senate is more moderate have support in Senators' tweets?**

**Note**: See the project handout for more details on caveats and the data dictionary.

In [10]:
# pandas and numpy
import pandas as pd
import numpy as numpy

# punctuation, stop words and English language model
from string import punctuation
from spacy.tokens import DocBin
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
nlp = en_core_web_sm.load()

# textblob
from textblob import TextBlob

# countvectorizer, tfidfvectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import LabelBinarizer

# gensim
import gensim
from gensim import models

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import scattertext as st
import pickle
import polars as pl
import pyarrow

In [ ]:
# load data 
# ----------
congress_tweets = pd.read_csv("../../data/116th Congressional Tweets and Demographics.csv")
# fill in this line of code with a sufficient number of tweets, depending on your computational resources
#congress_tweets = congress_tweets.sample(...)
congress_tweets.head()

## Preprocessing

The first step in working with text data is to preprocess it. Make sure you do the following:

* Remove punctuation and stop words. The `rem_punc_stop()` function we used in lab is provided to you but you should feel free to edit it as necessary for other steps
* Remove tokens that occur frequently in tweets, but may not be helpful for downstream classification. For instance, many tweets contain a flag for retweeting, or share a URL 

As you search online, you might run into solutions that rely on regular expressions. You are free to use these, but you should also be able to preprocess using the techniques we covered in lab. Specifically, we encourage you to use spaCy's token attributes and string methods to do some of this text preprocessing.

In [ ]:
def pre_model(text):
    
    punc = set(punctuation)
    
    punc_free = "".join([ch for ch in text if ch not in punc])
    return punc_free

def post_model(doc):
    stop_words = STOP_WORDS
    spacy_words = [token.text for token in doc if not token.like_url]
    
    spacy_words = [word for word in spacy_words if not (word.startswith('http') or word.startswith("RT") or word.startswith("QT"))]
    
    no_punc = [word for word in spacy_words if word not in stop_words]
    
    return no_punc

In [ ]:
#pre_model_data = congress_tweets['text'].map(pre_model)

In [ ]:
#docs = list(nlp.pipe(pre_model_data,n_process=4, batch_size=4000))

In [ ]:
# doc_bin = DocBin(docs=docs)
# doc_bin.to_disk("./data.spacy")
doc_bin = DocBin().from_disk("./data.spacy")
new_docs = list(doc_bin.get_docs(nlp.vocab))

In [ ]:
# post_model_data = [post_model(doc) for doc in new_docs]
# congress_tweets['tokens'] = post_model_data
# congress_tweets['joined_tokens'] = congress_tweets['tokens'].map(lambda text: ' '.join(text))

In [ ]:
# congress_tweets.head()
# congress_tweets.to_pickle('cleanedData.pkl')
congress_tweets = pd.read_pickle("cleanedData.pkl") 

In [ ]:
congress_tweets['party'].unique()

## Exploratory Data Analysis

Use two of the techniques we covered in lab (or other techniques outside of lab!) to explore the text of the tweets. You should construct these visualizations with an eye toward the eventual classification tasks: (1) predicting the legislator's political party based on the text of their tweet, and (2) predicting whether the legislator is a Senator or Representative. As a reminder, in lab we covered word frequencies, word clouds, word/character counts, scattertext, and topic modeling as possible exploration tools. 

### EDA 1

In [ ]:
wordcloud_text = ' '.join(congress_tweets['joined_tokens'])
wordcloud = WordCloud(background_color="white",random_state=111).generate(wordcloud_text)

In [ ]:
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

### EDA 2

In [ ]:

scatter_tweets = congress_tweets
scatter_tweets['doc'] = new_docs
corpus = st.CorpusFromParsedDocuments(congress_tweets, category_col="party", parsed_col="doc").build()

In [ ]:
html = st.produce_scattertext_explorer(corpus,
                                       category='Democrat',
                                       category_name='Democratic',
                                       not_category_name='Republican',
                                       width_in_pixels=1000,
                                       minimum_term_frequency=5,
                                       metadata=congress_tweets['screen_name'])

In [ ]:
open("PoliticalTweets.html", 'wb').write(html.encode('utf-8'))

## Sentiment Analysis

Next, let's analyze the sentiments contained within the tweets. You may use TextBlob or another library for these tasks. Do the following:

* Choose two legislators, one who you think will be more liberal and one who you think will be more conservative, and analyze their sentiment and/or subjectivity scores per tweet. For instance, you might do two scatterplots that plot each legislator's sentiment against their subjectivity, or two density plots for their sentiments. Do the scores match what you thought?
* Plot two more visualizations like the ones you chose in the first part, but do them to compare (1) Democrats v. Republicans and (2) Senators v. Representatives 

`TextBlob` has already been imported in the top cell.

In [24]:
# congress_tweets['polarity'] = congress_tweets['joined_tokens'].map(lambda x: TextBlob(x).sentiment.polarity)
# congress_tweets['subjectivity'] = congress_tweets['joined_tokens'].map(lambda x: TextBlob(x).sentiment.subjectivity)
#congress_tweets.drop(columns=['doc']).to_pickle('sentimentData.pkl')
congress_tweets = pd.read_pickle("sentimentData.pkl") 
tweets = pl.from_pandas(congress_tweets)
tweets = tweets.filter((pl.col('party') == "Republican") | (pl.col("party") == "Democrat"))
tweets.head()

tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share,tokens,joined_tokens,polarity,subjectivity
f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,list[str],str,f64,f64
1.0810e18,"""RepByrne""","""2019-01-03T21:23:00-05:00""","""Great news for Baldwin County!…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Great"", ""news"", … "" ""]","""Great news Baldwin County The …",0.6,0.65
1.0809e18,"""RepByrne""","""2019-01-03T12:30:38-05:00""","""Outstanding news today from @A…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Outstanding"", ""news"", … "" ""]","""Outstanding news today Airbus …",0.5,0.75
1.0808e18,"""RepByrne""","""2019-01-03T09:12:07-05:00""","""RT @senatemajldr Democrats wil…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""senatemajldr"", ""Democrats"", … ""solve""]","""senatemajldr Democrats border …",0.0,0.0
1.0809e18,"""RepByrne""","""2019-01-03T13:20:53-05:00""","""Here is a sign of things to co…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Here"", ""sign"", … ""Congress""]","""Here sign things come As Democ…",-0.024242,0.413636
1.0809e18,"""RepByrne""","""2019-01-03T12:10:26-05:00""","""Let's understand what we're de…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Lets"", ""understand"", … ""security""]","""Lets understand dealing border…",0.0,0.0


In [ ]:
two_sen = tweets.filter(pl.col("name_wikipedia").is_in(["Ted Cruz", "Maxine Waters"])).select("name_wikipedia","subjectivity","polarity","tweet_id")
sns.scatterplot(
   two_sen,
    x="subjectivity",
    y="polarity",
    hue="name_wikipedia",
    alpha=.3
)
plt.show()


In [ ]:
sns.kdeplot(two_sen,
            x="polarity",
            hue="name_wikipedia",
            )
plt.show()
sns.kdeplot(two_sen,
            x="subjectivity",
            hue="name_wikipedia"
            )
plt.show()

In [ ]:
sns.scatterplot(tweets.select(["tweet_id","party","polarity","subjectivity"]),
            x="polarity",
            y="subjectivity",
            hue="party",
            alpha=.1
            )
plt.show()

In [ ]:
sns.kdeplot(tweets.select(["tweet_id","party","polarity","subjectivity"]),
            x="polarity",
            hue="party",
            )
plt.show()

## Featurization

Before going to classification, explore different featurization techniques. Create three dataframes or arrays to represent your text features, specifically:

* Features engineered from your previous analysis. For example, word counts, sentiment scores, topic model etc.
* A term frequency-inverse document frequency matrix. 
* An embedding-based featurization (like a document averaged word2vec)

In the next section, you will experiment with each of these featurization techniques to see which one produces the best classifications.

### Bag-of-words or Tf-idf

In [26]:
tf = TfidfVectorizer()
token_list = pl.Series(tweets.select('joined_tokens')).to_list()

In [27]:
# Frequency Based featurization
tfidf_matrix = tf.fit_transform(token_list)

In [28]:
def print_top_words(model, feature_names, n_top_words):   # define function and parameters
    for topic_idx, topic in enumerate(model.components_): # iterate over each topic 
        print("\nTopic #{}:".format(topic_idx))           # print topic index
        print(" ".join([feature_names[i]                  # print topics
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [29]:
lda = LatentDirichletAllocation(n_components=5,
                                max_iter=1,
                                random_state=0).fit(tfidf_matrix)
tf_feature_names = tf.get_feature_names_out()

# apply function to print top 20 words
print_top_words(lda,                # model 
                tf_feature_names,   # feature names 
                20)    


Topic #0:
amp the today we this house act health care covid19 americans people need work it trump american help bill new

Topic #1:
amp we the today health help trump people president work covid19 senate this americans act need care house us time

Topic #2:
amp we the time today need president trump act this work american covid19 workers help health thank families new house

Topic #3:
amp the today this president we trump it house need congress people thank day bill support health work american america

Topic #4:
amp the today people we this act covid19 trump coronavirus american new need house president congress health great national bill



In [30]:
topic_dist = lda.transform(tfidf_matrix)

### Engineered Text Features

In [ ]:
engineered_features = tweets.with_columns(
    word_count= (pl.col('tokens').list.len()),
    length=(pl.col('joined_tokens').str.len_chars())
    ).select(["word_count","length","polarity","subjectivity"])

### Word Embedding

In [ ]:
# Load Word2Vec model from Google; OPTIONAL depending on your computational resources (the file is ~1 GB)
# Also note that this file path assumes that the word vectors are underneath 'data'; you may wish to point to the CSS course repo and change the path
# or move the vector file to the project repo 

#model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary = True) 

FileNotFoundError: [Errno 2] No such file or directory: 'data/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
# Function to average word embeddings for a document; use examples from lab to apply this function. You can use also other techniques such as PCA and doc2vec instead.
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

In [ ]:
# embedding based featurization


In [11]:
tweets.head()

tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share,tokens,joined_tokens,polarity,subjectivity
f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,list[str],str,f64,f64
1.0810e18,"""RepByrne""","""2019-01-03T21:23:00-05:00""","""Great news for Baldwin County!…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Great"", ""news"", … "" ""]","""Great news Baldwin County The …",0.6,0.65
1.0809e18,"""RepByrne""","""2019-01-03T12:30:38-05:00""","""Outstanding news today from @A…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Outstanding"", ""news"", … "" ""]","""Outstanding news today Airbus …",0.5,0.75
1.0808e18,"""RepByrne""","""2019-01-03T09:12:07-05:00""","""RT @senatemajldr Democrats wil…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""senatemajldr"", ""Democrats"", … ""solve""]","""senatemajldr Democrats border …",0.0,0.0
1.0809e18,"""RepByrne""","""2019-01-03T13:20:53-05:00""","""Here is a sign of things to co…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Here"", ""sign"", … ""Congress""]","""Here sign things come As Democ…",-0.024242,0.413636
1.0809e18,"""RepByrne""","""2019-01-03T12:10:26-05:00""","""Let's understand what we're de…","""Bradley Byrne""","""Rep""","""8-Jan-14""","""2/16/1955""","""M""","""AL""","""1""","""Republican""","""1,318,255""","""729,547""","""795,696""","""1,255,925""","[""Lets"", ""understand"", … ""security""]","""Lets understand dealing border…",0.0,0.0


## Classification

Either use cross-validation or partition your data with training/validation/test sets for this section. Do the following:

* Choose a supervised learning algorithm such as logistic regression, random forest etc. 
* Train six models. For each of the three dataframes you created in the featurization part, train one model to predict whether the author of the tweet is a Democrat or Republican, and a second model to predict whether the author is a Senator or Representative.
* Report the accuracy and other relevant metrics for each of these six models.
* Choose the featurization technique associated with your best model. Combine those text features with non-text features. Train two more models: (1) A supervised learning algorithm that uses just the non-text features and (2) a supervised learning algorithm that combines text and non-text features. Report accuracy and other relevant metrics. 

If time permits, you are encouraged to use hyperparameter tuning or AutoML techniques like TPOT, but are not explicitly required to do so.

### Train Six Models with Just Text

In [ ]:
# six models ([engineered features, frequency-based, embedding] * [democrat/republican, senator/representative])
dataframes = [engineered_features]
techniques = ["Engineered Features"]

lb_style = LabelBinarizer()

targets = [lb_style.fit_transform(tweets.select(pl.col('party'))), 
           lb_style.fit_transform(tweets.select(pl.col('position')))]

target_labels = ["Republican/Democrat","Senator/Representative"]

In [ ]:
for dataframe, technique in zip(dataframes, techniques):
    for target, target_label in zip(targets,target_labels):
            X_train, X_test, y_train, y_test = train_test_split(dataframe,   
                                                        target,
                                                        train_size = .80,
                                                        test_size=0.20,
                                                        random_state = 10)
            logit_reg = LogisticRegression()
            logit_model = logit_reg.fit(X_train,
                                        y_train.ravel())
            
            y_pred = logit_model.predict(X_test)

            cf_matrix = confusion_matrix(y_test,y_pred,normalize = 'true')

### Two Combined Models

In [ ]:
# two models ([best text features + non-text features] * [democrat/republican, senator/representative])
...

## Discussion Questions

1. Why do standard preprocessing techniques need to be further customized to a particular corpus?

**YOUR ANSWER HERE** ...

2. Did you find evidence for the idea that Democrats and Republicans have different sentiments in their tweets? What about Senators and Representatives?

**YOUR ANSWER HERE** ...

3. Why is validating your exploratory and unsupervised learning approaches with a supervised learning algorithm valuable?

**YOUR ANSWER HERE** ...

4. Did text only, non-text only, or text and non-text features together perform the best? What is the intuition behind combining text and non-text features in a supervised learning algorithm?

**YOUR ANSWER HERE** ...